In [6]:
import pandas as pd
import numpy as np
import MICE
import iMICE

#convert nominal
def processAge(x):
    if x >= 0 and x < 2:
        return 0
    elif x >= 2 and x <= 12:
        return 1
    elif x >= 13 and x <= 59:
        return 2
    elif x >= 60:
        return 3

df = pd.read_csv('titanic.csv')
df = df.drop(df.columns[[10]],axis = 1)
df = df.dropna(axis=0)
df = df.reset_index(drop = True)
df_1=df.copy()
df_1['AgeCode'] = df['Age'].map(processAge)

X = df_1.iloc[:, [2,9]].values  
y = df_1.iloc[:, 11].values

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

#actual
actual=np.concatenate((y_train,y_test),axis=0)
r1=len(y_train)

#y_test
df_2n = pd.DataFrame({'Age':y_test[:]})
df_2n.at[:,'Age']=np.nan
y_test_n=df_2n.iloc[:,0].values

y_train_1=np.concatenate((y_train,y_test_n),axis=0)
X_train_1=np.concatenate((X_train,X_test),axis=0)

df_2 = pd.DataFrame({'Age':y_train_1[:],'Pclass':X_train_1[:,0],'Fare':X_train_1[:,1]})

mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

print(mice_result.shape)
df_3 = pd.DataFrame(mice_result, columns = ['Age', 'Pclass', 'Fare'])
predict = df_3['Age']

predict = np.array(predict)

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predict[r1:]))
print(confusion_matrix(actual[r1:], predict[r1:]))

(712, 3)
0.8461538461538461
[[  0   0   2   0]
 [  0   0  10   0]
 [  0   3 121   0]
 [  0   1   6   0]]


# iMICE

In [9]:
imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3i = pd.DataFrame(imice_result, columns = ['Age Binary', 'Pclass', 'Fare'])
predicti = df_3i['Age Binary'].map(lambda x: int(x))

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix  
print(accuracy_score(y_test, predicti[r1:]))
print(confusion_matrix(actual[r1:], predicti[r1:]))

0.8671328671328671
[[  0   0   2   0]
 [  0   0  10   0]
 [  0   0 124   0]
 [  0   0   7   0]]


# KNN

In [12]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(3, weights='distance')
trained_model = clf.fit(X_train, y_train)
y_pred = trained_model.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8531468531468531
[[  1   0   1   0]
 [  0   4   6   0]
 [  0   7 117   0]
 [  0   0   7   0]]


# Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

regressor = RandomForestClassifier(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test)

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8461538461538461
[[  0   1   1   0]
 [  0   2   8   0]
 [  0   5 119   0]
 [  0   0   7   0]]
